In [1]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.6 MB/s eta 0:00:00


In [2]:
import tempfile
import os

import tensorflow as tf

from tensorflow_model_optimization.python.core.keras.compat import keras

# CIFAR 10 without quantization aware training

In [5]:
# Load CIFAR 10 dataset
cifar = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(10)  # Output layer with 10 classes
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

1407/1407 [==============================] - 23s 6ms/step - loss: 1.5031 - accuracy: 0.4521 - val_loss: 1.2033 - val_accuracy: 0.5728


In [10]:
model.fit(
  train_images,
  train_labels,
  epochs=10,
  validation_split=0.1,
)

Epoch 1/10
1407/1407 [==============================] - 7s 5ms/step - loss: 0.1112 - accuracy: 0.9623 - val_loss: 2.1090 - val_accuracy: 0.7124
Epoch 2/10
1407/1407 [==============================] - 9s 6ms/step - loss: 0.1156 - accuracy: 0.9608 - val_loss: 2.1206 - val_accuracy: 0.7142
Epoch 3/10
1407/1407 [==============================] - 6s 5ms/step - loss: 0.1003 - accuracy: 0.9661 - val_loss: 2.2440 - val_accuracy: 0.6942
Epoch 4/10
1407/1407 [==============================] - 7s 5ms/step - loss: 0.1137 - accuracy: 0.9623 - val_loss: 2.1396 - val_accuracy: 0.7084
Epoch 5/10
1407/1407 [==============================] - 6s 4ms/step - loss: 0.0987 - accuracy: 0.9663 - val_loss: 2.2766 - val_accuracy: 0.6998
Epoch 6/10
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1056 - accuracy: 0.9652 - val_loss: 2.2273 - val_accuracy: 0.7058
Epoch 7/10
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0936 - accuracy: 0.9687 - val_loss: 2.3879 - val_accuracy:

## Clone and fine-tune pre-trained model with quantization aware training


In [11]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (Quantize  (None, 32, 32, 3)         3         
 Layer)                                                          
                                                                 
 quant_conv2d_3 (QuantizeWr  (None, 30, 30, 32)        963       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_3 (Qua  (None, 15, 15, 32)        1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_conv2d_4 (QuantizeWr  (None, 13, 13, 64)        18627     
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_4 (Qua  (None, 6, 6, 64)         

### Train and evaluate the model against baseline

To demonstrate fine tuning after training the model for just an epoch, fine tune with quantization aware training on a subset of the training data.

In [12]:
train_images_subset = train_images[0:60000] # out of 60000
train_labels_subset = train_labels[0:60000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=1000, epochs=50, validation_split=0.01)

Epoch 1/50
50/50 [==============================] - 10s 66ms/step - loss: 0.2568 - accuracy: 0.9272 - val_loss: 1.7764 - val_accuracy: 0.6900
Epoch 2/50
50/50 [==============================] - 2s 44ms/step - loss: 0.1477 - accuracy: 0.9681 - val_loss: 1.8825 - val_accuracy: 0.7000
Epoch 3/50
50/50 [==============================] - 2s 44ms/step - loss: 0.1123 - accuracy: 0.9768 - val_loss: 1.9695 - val_accuracy: 0.7080
Epoch 4/50
50/50 [==============================] - 2s 41ms/step - loss: 0.0858 - accuracy: 0.9832 - val_loss: 1.9914 - val_accuracy: 0.7140
Epoch 5/50
50/50 [==============================] - 2s 41ms/step - loss: 0.0640 - accuracy: 0.9883 - val_loss: 1.9412 - val_accuracy: 0.7140
Epoch 6/50
50/50 [==============================] - 2s 42ms/step - loss: 0.0490 - accuracy: 0.9909 - val_loss: 2.0309 - val_accuracy: 0.7200
Epoch 7/50
50/50 [==============================] - 2s 42ms/step - loss: 0.0370 - accuracy: 0.9934 - val_loss: 2.0650 - val_accuracy: 0.7160
Epoch 8/50
5

For this example, there is minimal to no loss in test accuracy after quantization aware training, compared to the baseline.

In [13]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.6876999735832214
Quant test accuracy: 0.7235999703407288


In [19]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [25]:
# Save the converted model to a file
with open('NewQNNmodel.tflite', 'wb') as f:
    f.write(quantized_tflite_model)


In [17]:
# Create float TFLite model.
float_converter = tf.lite.TFLiteConverter.from_keras_model(model)
float_tflite_model = float_converter.convert()

# Measure sizes of models.
_, float_file = tempfile.mkstemp('.tflite')
_, quant_file = tempfile.mkstemp('.tflite')

with open(quant_file, 'wb') as f:
  f.write(quantized_tflite_model)

with open(float_file, 'wb') as f:
  f.write(float_tflite_model)

print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))

Float model in Mb: 0.8702888488769531
Quantized model in Mb: 0.2286376953125
